<a href="https://colab.research.google.com/github/CE339/CES-S24/blob/main/Module02/RiverpolConcreteModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Initializing Google Colab**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
os.chdir('/content/drive/MyDrive/Colab Notebooks/')

**Installing Pyomo and a solver**. We will use glpk as a solver for this model. Next time we will install and use COIN-OR CBC.

In [ ]:
!pip install pyomo
!apt-get install -y -qq glpk-utils
#!apt-get install -y -qq coinor-cbc

**Importing Pyomo and CBC Solver**

In [ ]:
from pyomo.environ import *
#Import solver
opt=SolverFactory('glpk',executable='/usr/bin/glpk')

**Building the concrete model**

**In this exercise we will create a Concrete model in Pyomo and solve it**.
In many contexts, a mathematical model can and should be directly defined with the data values supplied at the time of the model definition. We call these concrete mathematical models. For example, the following LP model can be represented in Pyomo as a concrete model:

      min  2x1 + 5x2
      s.t. 5x1 + 3x2
       x1, x2 >= 0

The ConcreteModel class is used to define concrete optimization models in Pyomo. We will explore this using the River Pollution example using Sets.

In [ ]:
from math import inf

model = ConcreteModel()

model.I = Set(initialize=["F_A","F_B"]) # Set of Factories
model.J = Set(initialize=['P_1','P_2']) # Set of Pollutants

# Parameters Definition
# ci - Cost to process waste from factory
model.c    = Param(model.I, initialize={"F_A":15 , "F_B":10})
# rij - Rate of reduction in pollutant type if emission is treated at factory
model.r    = Param(model.I, model.J, initialize={("F_A",'P_1'):0.1, ("F_A",'P_2'):0.4,("F_B",'P_1'):0.2,("F_B",'P_2'):0.16})
# Sj - State requirement to reduce pollutants
model.S = Param(model.J,initialize={'P_1':30, 'P_2':40})

# Decision Variables Definition
model.w = Var(model.I, within=NonNegativeReals) # wi - Processed waste from Factory i

# Objective Function - Minimize the cost of waste treatment to reduce pollutants
def cost_rule(model):
    return sum(model.c[i]*model.w[i] for i in model.I)
model.cost = Objective(rule=cost_rule)

# Constraints - Satisfy the state's requeriments on pollutant reduction
def pollutant_rule(model, j):
    return sum(model.r[i,j]*model.w[i] for i in model.I) >= model.S[j]
model.pollutant = Constraint(model.J, rule=pollutant_rule)

**Optimizing the concrete model**

In [ ]:
opt = SolverFactory('glpk')
results = opt.solve(model, tee=True)

**Display Results**

In [ ]:
print(model.display())